# Grid World迷宫之策略循环(Policy Iteration)

In [1]:
# 载入相关套件
import numpy as np
from lib.envs.gridworld import GridworldEnv

In [2]:
# 环境
env = GridworldEnv()

In [3]:
# 策略评估函数
def policy_eval(policy, env, discount_factor=1.0, theta=0.00001):
    # 状态值函数初始化
    V = np.zeros(env.nS)
    while True:
        delta = 0
        # 更新每个状态值的函数
        for s in range(env.nS):
            v = 0
            # 计算每个行动后的状态值函数
            for a, action_prob in enumerate(policy[s]):
                # 取得所有可能的下一状态值
                for  prob, next_state, reward, done in env.P[s][a]:
                    # 状态值函数公式，依照所有可能的下一状态值函数加总 
                    v += action_prob * prob * (reward + discount_factor * V[next_state])
            # 比较更新前后的差值，取最大值
            delta = max(delta, np.abs(v - V[s]))
            V[s] = v
        # 若最大差值 < 门槛值，则停止评估
        if delta < theta:
            break
    return np.array(V)

In [4]:
# 策略改善函数
def policy_improvement(env, policy_eval_fn=policy_eval, discount_factor=1.0):
    # 计算行动值函数
    def one_step_lookahead(state, V):
        A = np.zeros(env.nA)
        for a in range(env.nA):
            for prob, next_state, reward, done in env.P[state][a]:
                A[a] += prob * (reward + discount_factor * V[next_state])
        return A
    
    # 一开始采随机策略，往上/下/左/右走的机率(π)均等
    policy = np.ones([env.nS, env.nA]) / env.nA
    
    while True:
        # 策略评估
        V = policy_eval_fn(policy, env, discount_factor)
        
        # 若要改变策略，会设定 policy_stable = False
        policy_stable = True
        
        for s in range(env.nS):
            # 依 P 选择最佳行动
            chosen_a = np.argmax(policy[s])
            
            # 计算下一步的行动值函数
            action_values = one_step_lookahead(s, V)
            # 选择最佳行动
            best_a = np.argmax(action_values)
            
            # 贪婪策略：若有新的最佳行动，修改行动策略
            if chosen_a != best_a:
                policy_stable = False
            policy[s] = np.eye(env.nA)[best_a]
        
        # 如果已无较佳行动策略，则回传策略及状态值函数
        if policy_stable:
            return policy, V

In [5]:
# 执行策略循环
policy, v = policy_improvement(env)

In [9]:
# 显示结果
print("策略机率分配:")
print(policy)
print("")

print("4x4 策略机率分配 (0=up, 1=right, 2=down, 3=left):")
print(np.reshape(np.argmax(policy, axis=1), env.shape))
print("")

print("状态值函数:")
print(v)
print("")

print("4x4 状态值函数:")
print(v.reshape(env.shape))
print("")

策略机率分配:
[[1. 0. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]]

4x4 策略机率分配 (0=up, 1=right, 2=down, 3=left):
[[0 3 3 2]
 [0 0 0 2]
 [0 0 1 2]
 [0 1 1 0]]

状态值函数:
[ 0. -1. -2. -3. -1. -2. -3. -2. -2. -3. -2. -1. -3. -2. -1.  0.]

4x4 状态值函数:
[[ 0. -1. -2. -3.]
 [-1. -2. -3. -2.]
 [-2. -3. -2. -1.]
 [-3. -2. -1.  0.]]



In [7]:
# 验证答案是否正确
expected_v = np.array([ 0, -1, -2, -3, -1, -2, -3, -2, -2, -3, -2, -1, -3, -2, -1,  0])
np.testing.assert_array_almost_equal(v, expected_v, decimal=2)